# Tutorial 4 - Segmentation (Test)

## Open Dataset : Lumbar vertebra segmentation CT image database

In [1]:
## Python 3.6
from __future__ import absolute_import, division, print_function
import os
import glob
import numpy as np
import h5py
import cv2
import matplotlib.pyplot as plt


import keras
from keras.models import *
from keras.layers import Input, merge, BatchNormalization, AtrousConvolution2D, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, concatenate
from keras.layers.advanced_activations import PReLU

from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import  ModelCheckpoint, CSVLogger, ReduceLROnPlateau, LearningRateScheduler

from models.custom_model import UNet_like


/Users/dhlee/anaconda/envs/tensorflow3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Parameter Setting

In [2]:
#############################################
prediction_name = 'UNet_like2_epoch100_acc.npy'
model_name = "UNet_like2_epoch100_acc.hdf5" # UNet_like_epoch100.hdf5 / UNet_like_epoch100_aug
#############################################

## Path Setting

In [3]:
npy_path = "../Dataset/Lumbar Vertebrae Semgnetation/npy/"
gt_path = '../Dataset/Lumbar Vertebrae Semgnetation/Test/mask/'
pred_path = './Result/unet_result/'

save_path = os.path.join(os.getcwd(), 'Result/saved_model')
if not os.path.isdir(save_path):
    os.makedirs(save_path)    
model_path = os.path.join(save_path, model_name)

tf_graph_path = './Result/tf_graph/'    
if not os.path.isdir(tf_graph_path):
    os.makedirs(tf_graph_path)   
    
graph_path = './Result/graph_log/'    
if not os.path.isdir(graph_path):
    os.makedirs(graph_path)       
    
result_path = './Result/unet_result/'    
if not os.path.isdir(result_path):
    os.makedirs(result_path)    
    

## Load Test Dataset

In [4]:
## Image
image_test = np.load(npy_path + "image_test.npy")
image_test = image_test.astype('float32')
image_test /= 255

## Mask
mask_test = np.load(npy_path + "mask_test.npy")
mask_test = mask_test.astype('float32')
mask_test /= 255
mask_test[mask_test > 0.5] = 1
mask_test[mask_test <= 0.5] = 0

## Load Trained Model

In [6]:
#####################################################################################
#####################################################################################
trained_model = load_model('./Result/saved_model/' + model_name)
# load_weights
#####################################################################################
#####################################################################################

## Prediction

In [7]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
    
y_pred = trained_model.predict(image_test, batch_size=1, verbose=0)


""" Save (.npy) Optional """
np.save('npy_path' + prediction_name, y_pred)
      

""" Save (.png) """   
result_mask = np.load(npy_path + prediction_name)
for i in range(result_mask.shape[0]):
    result = array_to_img(result_mask[i])
    result.save(result_path + str(i+1).zfill(5) + '.png')
             

print("\n***** Prediction Complete *****")

KeyboardInterrupt: 

## Evaluation

In [ ]:
gt_list = []
for i, gt_img in enumerate(glob.glob(gt_path + '*.png')):
    gt = cv2.imread(gt_img)
    _, thresh_gt = cv2.threshold(gt,127,255, cv2.THRESH_BINARY)
    gt_list.append(thresh_gt)
    
pred_list = []
for i, pred_img in enumerate(glob.glob(pred_path + '*.png')):
    pred = cv2.imread(pred_img)
    _, thresh_pred = cv2.threshold(pred,127,255, cv2.THRESH_BINARY)
    pred_list.append(thresh_pred) 

    

""" Dice Similarity Coefficient """


## Homework : Evaulate using Dice Similarity Coefficient 